In [ ]:
# ============================================================================
# Extract Visual Features with ViT for CausalVidQA
# ============================================================================
# Mục tiêu:
# - Đọc video IDs từ file pkl (train.pkl, valid.pkl, test.pkl)
# - Trích xuất visual-feature bằng ViT `google/vit-large-patch16-224-in21k`
# - Lưu feature dưới dạng .h5 (giống format hiện tại của visual-feature)
# - Upload lên HuggingFace để dùng lại sau
#
# Output format (tương thích DataLoader.py):
# - vit_visual_feat.h5: chứa "resnet_features" shape (N_videos, T, D)
# - idx2vid.pkl: list video_id theo thứ tự index
# ============================================================================

# ============================================================================
# 0. Cài đặt thư viện
# ============================================================================
%pip install -q "transformers>=4.40.0" "huggingface_hub>=0.24.0" opencv-python tqdm h5py



In [ ]:
import os
import pickle
from pathlib import Path

import cv2
import h5py
import numpy as np
import torch
from tqdm.auto import tqdm
from transformers import AutoImageProcessor, AutoModel
from huggingface_hub import HfApi, login

# ============================================================================
# 1. CẤU HÌNH - SỬA LẠI CHO ĐÚNG DATASET CỦA BẠN
# ============================================================================

class Config:
    # ==================== KAGGLE PATHS ====================
    # Đường dẫn tới split files (train.pkl, valid.pkl, test.pkl)
    SPLIT_PATH = "/kaggle/input/casual-vid-data-split/split"
    
    # Đường dẫn tới thư mục chứa video gốc
    VIDEO_ROOT = "/kaggle/input/causalvid-videos"  # TODO: sửa lại
    
    # ==================== OUTPUT ====================
    # Thư mục lưu feature output
    OUTPUT_DIR = "/kaggle/working/vit_visual_features"
    
    # ==================== MODEL ====================
    VIT_NAME = "google/vit-large-patch16-224-in21k"
    
    # Số frame tối đa lấy mỗi video (lấy mẫu đều)
    NUM_FRAMES = 16
    
    # Thiết bị
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    
    # ==================== HUGGINGFACE ====================
    HF_TOKEN = ""          # Điền token để upload
    HF_REPO_ID = ""        # Ví dụ: "your-username/causalvid-vit-features"

cfg = Config()

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

print("=" * 60)
print("CONFIGURATION")
print("=" * 60)
print(f"SPLIT_PATH : {cfg.SPLIT_PATH}")
print(f"VIDEO_ROOT : {cfg.VIDEO_ROOT}")
print(f"OUTPUT_DIR : {cfg.OUTPUT_DIR}")
print(f"VIT_NAME   : {cfg.VIT_NAME}")
print(f"NUM_FRAMES : {cfg.NUM_FRAMES}")
print(f"DEVICE     : {cfg.DEVICE}")
print("=" * 60)


In [ ]:
# ============================================================================
# 2. LOAD VIDEO IDs TỪ PKL FILES
# ============================================================================

def load_pkl(path):
    with open(path, 'rb') as f:
        return pickle.load(f)

# Load tất cả video IDs từ train, valid, test
all_vids = set()

for split_name in ['train', 'valid', 'test']:
    split_file = os.path.join(cfg.SPLIT_PATH, f'{split_name}.pkl')
    if os.path.exists(split_file):
        vids = load_pkl(split_file)
        print(f"Loaded {len(vids)} videos from {split_name}.pkl")
        all_vids.update(vids)
    else:
        # Thử val.pkl thay vì valid.pkl
        alt_file = os.path.join(cfg.SPLIT_PATH, 'val.pkl')
        if split_name == 'valid' and os.path.exists(alt_file):
            vids = load_pkl(alt_file)
            print(f"Loaded {len(vids)} videos from val.pkl")
            all_vids.update(vids)
        else:
            print(f"[WARN] Not found: {split_file}")

all_vids = sorted(list(all_vids))
print(f"\n✅ Total unique videos: {len(all_vids)}")


In [ ]:
# ============================================================================
# 3. LOAD MÔ HÌNH ViT
# ============================================================================

print("Loading ViT model and processor ...")
processor = AutoImageProcessor.from_pretrained(cfg.VIT_NAME)
model = AutoModel.from_pretrained(cfg.VIT_NAME)
model.to(cfg.DEVICE)
model.eval()

HIDDEN_DIM = model.config.hidden_size  # 1024 cho vit-large

print(f"✅ Loaded {cfg.VIT_NAME}")
print(f"   Hidden size: {HIDDEN_DIM}")
print(f"   Image size : {getattr(model.config, 'image_size', 224)}")
print(f"   Patch size : {getattr(model.config, 'patch_size', 16)}")


In [ ]:
# ============================================================================
# 4. HÀM TIỆN ÍCH
# ============================================================================

def find_video_path(video_id, video_root):
    """Tìm file video theo video_id trong thư mục video_root"""
    video_root = Path(video_root)
    
    # Thử các extension phổ biến
    for ext in ['.mp4', '.avi', '.mkv', '.mov', '.webm']:
        # Thử trực tiếp
        path = video_root / f"{video_id}{ext}"
        if path.exists():
            return path
        
        # Thử trong subfolder
        for subpath in video_root.rglob(f"{video_id}{ext}"):
            return subpath
    
    return None


def load_video_frames(video_path, num_frames=16, target_size=(224, 224)):
    """Đọc video và lấy tối đa `num_frames` frame được lấy mẫu đều."""
    cap = cv2.VideoCapture(str(video_path))
    if not cap.isOpened():
        raise RuntimeError(f"Cannot open video: {video_path}")

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) or 0
    if total_frames == 0:
        cap.release()
        raise RuntimeError(f"Video has 0 frames: {video_path}")

    # Chọn index frame theo sampling đều
    num = min(num_frames, total_frames)
    indices = np.linspace(0, total_frames - 1, num=num, dtype=int)

    frames = []
    for idx in indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, int(idx))
        ok, frame_bgr = cap.read()
        if not ok:
            continue
        frame_rgb = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
        frame_rgb = cv2.resize(frame_rgb, target_size, interpolation=cv2.INTER_AREA)
        frames.append(frame_rgb)

    cap.release()

    if len(frames) == 0:
        raise RuntimeError(f"No frames decoded from: {video_path}")

    return frames


@torch.no_grad()
def extract_vit_features(video_path):
    """Trích xuất visual feature từ ViT [CLS] token.
    
    Returns:
        frame_feat: (T, D) - visual feature cho mỗi frame
    """
    # Lấy target size từ processor
    if isinstance(processor.size, dict) and "shortest_edge" in processor.size:
        target_size = (processor.size["shortest_edge"],) * 2
    else:
        target_size = (224, 224)
    
    frames = load_video_frames(video_path, num_frames=cfg.NUM_FRAMES, target_size=target_size)

    # Chuẩn bị batch tensor
    inputs = processor(images=frames, return_tensors="pt")
    inputs = {k: v.to(cfg.DEVICE) for k, v in inputs.items()}

    outputs = model(**inputs)
    hidden = outputs.last_hidden_state  # (T, 1 + P, D)

    # Token [CLS] là token đầu tiên -> visual feature của frame
    cls_tok = hidden[:, 0]  # (T, D)
    frame_feat = cls_tok.detach().cpu().numpy()

    return frame_feat.astype(np.float32)

print("✅ Helper functions defined")


In [ ]:
# ============================================================================
# 5. TRÍCH XUẤT FEATURES CHO TẤT CẢ VIDEOS
# ============================================================================

print("=" * 60)
print("EXTRACTING ViT FEATURES")
print("=" * 60)

# Chuẩn bị lưu trữ
features_dict = {}  # video_id -> (T, D)
idx2vid = []        # index -> video_id
failed_vids = []    # videos không extract được

for vid in tqdm(all_vids, desc="Extracting ViT features"):
    video_path = find_video_path(vid, cfg.VIDEO_ROOT)
    
    if video_path is None:
        failed_vids.append((vid, "Video file not found"))
        continue
    
    try:
        frame_feat = extract_vit_features(video_path)
        features_dict[vid] = frame_feat
        idx2vid.append(vid)
    except Exception as e:
        failed_vids.append((vid, str(e)))
        continue

print(f"\n✅ Extracted features for {len(features_dict)} videos")
if failed_vids:
    print(f"❌ Failed: {len(failed_vids)} videos")
    for vid, err in failed_vids[:5]:
        print(f"   - {vid}: {err}")


In [ ]:
# ============================================================================
# 6. LƯU FEATURES DƯỚI DẠNG H5 (tương thích DataLoader.py)
# ============================================================================

print("=" * 60)
print("SAVING FEATURES")
print("=" * 60)

if len(features_dict) == 0:
    print("❌ No features to save!")
else:
    # Tìm số frame tối đa để pad
    max_frames = max(feat.shape[0] for feat in features_dict.values())
    feat_dim = HIDDEN_DIM
    n_videos = len(idx2vid)
    
    print(f"Number of videos: {n_videos}")
    print(f"Max frames: {max_frames}")
    print(f"Feature dim: {feat_dim}")
    
    # Tạo array (N, T, D) với padding
    all_features = np.zeros((n_videos, max_frames, feat_dim), dtype=np.float32)
    
    for i, vid in enumerate(idx2vid):
        feat = features_dict[vid]
        T = feat.shape[0]
        all_features[i, :T, :] = feat
    
    # Lưu h5 file
    h5_path = os.path.join(cfg.OUTPUT_DIR, "vit_visual_feat.h5")
    with h5py.File(h5_path, 'w') as f:
        f.create_dataset('resnet_features', data=all_features, compression='gzip')
    print(f"✅ Saved: {h5_path}")
    print(f"   Shape: {all_features.shape}")
    
    # Lưu idx2vid.pkl
    idx2vid_path = os.path.join(cfg.OUTPUT_DIR, "idx2vid.pkl")
    with open(idx2vid_path, 'wb') as f:
        pickle.dump(idx2vid, f)
    print(f"✅ Saved: {idx2vid_path}")
    print(f"   Videos: {len(idx2vid)}")
    
    # Lưu failed videos để debug
    if failed_vids:
        failed_path = os.path.join(cfg.OUTPUT_DIR, "failed_videos.txt")
        with open(failed_path, 'w') as f:
            for vid, err in failed_vids:
                f.write(f"{vid}\t{err}\n")
        print(f"⚠️ Saved failed list: {failed_path}")


In [ ]:
# ============================================================================
# 7. UPLOAD LÊN HUGGINGFACE HUB
# ============================================================================

if cfg.HF_TOKEN and cfg.HF_REPO_ID:
    print("=" * 60)
    print("UPLOADING TO HUGGINGFACE")
    print("=" * 60)
    
    login(token=cfg.HF_TOKEN)
    api = HfApi()
    
    # Tạo repo nếu chưa có
    api.create_repo(cfg.HF_REPO_ID, repo_type="dataset", exist_ok=True)
    
    # Upload toàn bộ thư mục output
    api.upload_folder(
        repo_id=cfg.HF_REPO_ID,
        folder_path=cfg.OUTPUT_DIR,
        repo_type="dataset",
    )
    
    print(f"✅ Uploaded to: https://huggingface.co/datasets/{cfg.HF_REPO_ID}")
else:
    print("\n[INFO] HF_TOKEN hoặc HF_REPO_ID chưa được cấu hình.")
    print("       Bỏ qua bước upload. Features đã được lưu local tại:")
    print(f"       {cfg.OUTPUT_DIR}")


In [ ]:
# ============================================================================
# 8. VERIFY - KIỂM TRA LẠI FEATURES ĐÃ LƯU
# ============================================================================

print("=" * 60)
print("VERIFICATION")
print("=" * 60)

h5_path = os.path.join(cfg.OUTPUT_DIR, "vit_visual_feat.h5")
idx2vid_path = os.path.join(cfg.OUTPUT_DIR, "idx2vid.pkl")

if os.path.exists(h5_path) and os.path.exists(idx2vid_path):
    # Load lại và kiểm tra
    with h5py.File(h5_path, 'r') as f:
        feats = f['resnet_features']
        print(f"✅ vit_visual_feat.h5")
        print(f"   Shape: {feats.shape}")
        print(f"   Dtype: {feats.dtype}")
        
        # Sample một video
        sample_feat = feats[0]
        print(f"   Sample video[0] shape: {sample_feat.shape}")
        print(f"   Sample video[0] mean: {sample_feat.mean():.4f}")
    
    with open(idx2vid_path, 'rb') as f:
        loaded_idx2vid = pickle.load(f)
    print(f"\n✅ idx2vid.pkl")
    print(f"   Number of videos: {len(loaded_idx2vid)}")
    print(f"   First 5 videos: {loaded_idx2vid[:5]}")
    
    print("\n" + "=" * 60)
    print("📋 SỬ DỤNG TRONG DATALOADER:")
    print("=" * 60)
    print(f"""
# Trong DataLoader.py, thay đổi đường dẫn:
video_feature_path = "{cfg.OUTPUT_DIR}"

# Hoặc sau khi upload lên HuggingFace:
# video_feature_path = "/path/to/downloaded/vit_visual_features"

# File structure:
#   {cfg.OUTPUT_DIR}/
#   ├── vit_visual_feat.h5   # (N, T, {HIDDEN_DIM})
#   └── idx2vid.pkl          # list of video_ids

# Lưu ý: Cần sửa DataLoader.py để load "vit_visual_feat.h5" 
# thay vì "appearance_feat.h5" và "motion_feat.h5"
""")
else:
    print("❌ Feature files not found!")


In [ ]:
# ============================================================================
# 9. SUMMARY
# ============================================================================

print("=" * 60)
print("✅ EXTRACTION COMPLETE!")
print("=" * 60)
print(f"""
📂 Output files:
   - {cfg.OUTPUT_DIR}/vit_visual_feat.h5
   - {cfg.OUTPUT_DIR}/idx2vid.pkl

📊 Feature info:
   - Model: {cfg.VIT_NAME}
   - Feature dim: {HIDDEN_DIM}
   - Frames per video: {cfg.NUM_FRAMES}

🔄 Để sử dụng features này trong training:
   1. Upload lên HuggingFace (đã làm nếu HF_TOKEN được cấu hình)
   2. Sửa DataLoader.py để load vit_visual_feat.h5
   3. Hoặc tạo DataLoader mới cho ViT features

🎯 Next steps:
   - Download features từ HuggingFace
   - Cập nhật DataLoader.py
   - Chạy training với ViT features
""")
